# 训练与生成
终于，我们来到了第四部分Training loop和Generating text，我们将最终整合迄今为止构建的主要组件：分词后的数据、模型和优化器。



## 5.1 数据加载（Data Loader）
分词后的数据（例如你在 tokenizer_experiments 中准备的数据）是一个单一的令牌序列 $x = (x_1, \cdots, x_n)$。尽管原始数据可能由多个独立的文档组成（例如不同的网页或源代码文件），常见的做法是将所有这些文档连接成一个单一的令牌序列，并在它们之间添加一个分隔符（例如 token）。

数据加载器会将这个长序列转换为一批批的数据流，其中每个批次包含 $B$ 个长度为 $m$ 的序列，以及对应的下一个令牌序列（长度也为 $m$）。例如，当 $B = 1,m = 3$ 时，$([x_2, x_3, x_4], [x_3, x_4, x_5])$ 就是一个可能的批次。

以这种方式加载数据在多个方面简化了训练过程。首先，任何满足 1 ≤ i < n − m 的位置 i 都可以生成一个有效的训练序列，因此序列的采样非常简单。由于所有训练序列长度相同，无需进行填充操作，这提高了硬件利用率（同时也可以增大批次大小 B）。

最后，我们也不必一次性将整个数据集全部加载到内存中即可进行训练样本的抽取，从而更容易处理那些可能无法完全放入内存的大型数据集。

#### 问题（data_loading）：实现数据加载
交付要求：编写一个函数，该函数接收一个 numpy 数组 x（包含令牌 ID 的整数数组）、batch_size、context_length 和一个 PyTorch 设备字符串（例如 'cpu' 或 'cuda:0'），并返回一对张量：采样的输入序列及其对应的下一个令牌目标。这两个张量的形状都应为 (batch_size, context_length)，包含令牌 ID，并且都应放置在指定的设备上。

> **低资源/降级训练提示：在 CPU 上进行数据加载**
> 
>如果你计划在 CPU 上训练你的语言模型，你需要将数据移动到正确的设备上（同样地，之后你的模型也应使用相同的设备）。

* 如果使用 CPU，可使用设备字符串 'cpu'；

In [1]:
import numpy as np
import torch


def get_batch(
    x: np.ndarray, batch_size: int, context_length: int, device: str
) -> tuple[torch.Tensor, torch.Tensor]:
    """
    生成训练批次数据

    Args:
        x: 令牌ID的一维numpy数组
        batch_size: 批次大小
        context_length: 序列长度
        device: 目标设备 (如 'cpu' 或 'cuda:0')

    Returns:
        (inputs,targets): (输入序列, 下一个目标序列) (batch_size, context_length)
    """
    # 确保数据足够长 (至少需要 context_length+1 个令牌)
    assert len(x) > context_length + 1, "数据长度不足"

    # 随机生成起始位置 (避免越界)
    max_start = len(x) - context_length - 1
    starts = np.random.randint(0, max_start, size=batch_size)

    # 提取输入序列和目标序列,生成批次 (batch_size, context_length)
    inputs = np.stack([x[i : i + context_length] for i in starts])
    targets = np.stack([x[i + 1 : i + context_length + 1] for i in starts])

    # 转为PyTorch张量并移动到指定设备
    return (
        torch.tensor(inputs, dtype=torch.long, device=device),
        torch.tensor(targets, dtype=torch.long, device=device),
    )

# 生成示例数据 (1000个令牌ID)
data = np.random.randint(0, 100, size=1000)

# 获取批次 (batch_size=4, 序列长=8, 使用CPU)
inputs, targets = get_batch(data, 4, 8, 'cpu')

print(inputs.shape)  # torch.Size([4, 8])
print(targets.shape) # torch.Size([4, 8])
print(inputs.device) # cpu


torch.Size([4, 8])
torch.Size([4, 8])
cpu


如果数据集太大而无法全部加载到内存中怎么办？我们可以使用一个名为 mmap 的 Unix 系统调用，它能将磁盘上的文件映射到虚拟内存，并在访问对应内存位置时才懒加载文件内容。因此，你可以“假装”整个数据集已经在内存中了。NumPy 通过 np.memmap 提供了这一功能（或者在使用 np.load 时设置参数 mmap_mode='r'，前提是你之前是用 np.save 保存的数组），这会返回一个类数组对象，只有在你访问具体元素时才会按需加载数据。

在训练过程中从数据集（即 NumPy 数组）采样时，务必以内存映射模式加载数据集（通过 np.memmap 或 np.load 的 mmap_mode='r' 参数，具体取决于你保存数组的方式）。同时，请确保指定的 dtype 与你所加载的数组的原始数据类型一致。为确保安全，建议显式验证内存映射的数据是否正确（例如，检查数据中是否包含超出预期词表大小的非法 token ID 值）。

评论
